In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 用于 Python 的 Vertex AI SDK：AutoML 训练层次预测用于批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_forecasting_hierarchical_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_forecasting_hierarchical_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_forecasting_hierarchical_batch.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程演示了如何使用 Vertex AI SDK for Python 创建层次预测模型，并使用 Google Cloud 的 [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) 进行批量预测。具体来说，您将根据历史销售数据预测一个虚构商店的销售额。

了解更多关于[表格数据的层次预测](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/hierarchical)。

### 目标

在本教程中，您将使用 Vertex AI SDK for Python 创建一个 AutoML 分层预测模型，并部署它进行批量预测。您也可以使用`gcloud`命令行工具创建和部署模型，或者使用 Cloud Console 进行批量操作。
分层预测模型的原理是为了在给定的销售数据组中最小化错误。在本教程中，您将在“产品”级别上最小化销售预测的误差。

本教程使用以下 Google Cloud ML 服务：

- `AutoML 训练`
- `Vertex AI 数据集`

执行的步骤包括：

- 创建一个 Vertex AI `TimeSeriesDataset` 资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

### 数据集

本教程使用的数据集是一个合成生成的虚构户外装备商店销售数据集。在此数据集中，您可以根据历史销售数据预测一家虚构商店的销售额。

此数据集是为模拟虚构户外装备商店的销售模式而合成的。以下是产品类别、产品类型和产品之间的层次关系：

- 产品类别：雪
    - 产品类型：滑雪板
        - 产品： 
        
此外，该公司有3个店铺位置，每个位置都有各自的客流量水平。
- 店铺：郊区
- 店铺：旗舰店
- 店铺：市中心

数据中还包含额外的季节效应。

数据链接：gs://cloud-samples-data/vertex-ai/structured_data/forecasting/synthetic_sales_data.csv

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

## 安装

安装最新版本的Cloud Storage、Bigquery和Vertex AI SDKs for Python。

In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        google-cloud-bigquery[pandas] \
                        seaborn \
                        scikit-learn

### 仅限Colab：取消注释以下单元格以重新启动核心。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

1. Vertex AI 工作台
* 无需操作，因为您已经通过认证。

本地JupyterLab实例 ，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何将云存储权限授予您的服务帐号：https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有当您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

初始化用于Python的Vertex AI SDK，为您的项目准备。

In [ ]:
from google.cloud import aiplatform

# Initialize the Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform

初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

# 教程

现在你已经准备好开始创建自己的AutoML时间序列预测模型了。

创建数据集

使用`TimeSeriesDataset.create()`来创建一个`TimeSeriesDataset`资源，需要以下参数：

- `display_name`：`Dataset`资源的人类可读的名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `bq_source`：或者，从BigQuery表导入数据项到`Dataset`资源。

此操作可能需要几分钟。

### 下载数据

In [ ]:
DATASET_URI = "gs://cloud-samples-data/vertex-ai/structured_data/forecasting/synthetic_sales_data.csv"

# Download the dataset
! gsutil cp {DATASET_URI} dataset.csv

拆分

在这个使用案例中，您需要预测每个产品每个店铺的销售量。
因此，您需要创建一个名为'product_at_store'的新列，这是'product'和'store'列的串联。这将在训练期间作为'target_column'传递。

最后，将数据集拆分为训练数据集和测试数据集。
训练数据集保存为CSV文件，但测试数据集需要进一步处理，比如移除目标列。

In [ ]:
import pandas as pd

DATASET_TRAIN_FILENAME = "sales_forecasting_train.csv"
DATASET_TEST_FILENAME = "sales_forecasting_test.csv"
DATASET_TRAIN_URI = f"{BUCKET_URI}/{DATASET_TRAIN_FILENAME}"

# Load dataset
df = pd.read_csv("dataset.csv")

df["date"] = df["date"].astype("datetime64[ns]")

# Add a target column
df["product_at_store"] = df["product"] + " (" + df["store"] + ")"

# Split dataset into train and test by taking the first 90% of data for training.
dates_unique = df["date"].unique()
date_cutoff = sorted(dates_unique)[round(len(dates_unique) * 9 / 10)]

# Save train dataset
df[df["date"] < date_cutoff].to_csv(DATASET_TRAIN_FILENAME, index=False)

# Create test dataset
df_test = df[df["date"] >= date_cutoff]

# Upload to GCS bucket
! gsutil cp {DATASET_TRAIN_FILENAME} {DATASET_TRAIN_URI}

绘制数据集

绘制“销售额”与“产品在店铺中”的图表，以了解数据集的情况。

请注意冬季月份“雪”产品销量的高峰期，以及夏季月份“水”产品销量的高峰期。

In [ ]:
import seaborn as sns

sns.relplot(
    data=df,
    x="date",
    y="sales",
    hue="product_at_store",
    row="product_category",
    aspect=4,
    kind="line",
    style="store",
)

In [ ]:
dataset_time_series = aiplatform.TimeSeriesDataset.create(gcs_source=DATASET_TRAIN_URI)

print(dataset_time_series.resource_name)

### 创建并运行训练管道

要训练一个自动机器学习模型，请创建并运行一个训练管道。

#### 创建训练任务

使用`AutoMLForecastingTrainingJob`类创建一个自动机器学习训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `column_transformations`：（可选）要应用于输入列的变换。
- `optimization_objective`：优化目标（最小化或最大化）。
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name="hierachical_sales_forecasting",
    optimization_objective="minimize-rmse",
    column_specs={
        "date": "timestamp",
        "sales": "numeric",
        "product_type": "categorical",
        "product_category": "categorical",
        "product": "categorical",
        "store": "categorical",
    },
)

### 设置上下文和预测范围

在训练预测模型时，您需要考虑上下文窗口和预测范围。
- 上下文窗口设置了模型在训练期间（以及预测期间）查看数据的时间范围。换句话说，对于每个训练数据点，上下文窗口决定了模型查找预测模式的时间范围。
- 预测范围确定了模型预测每个预测数据行的目标值的未来时间范围。

请参阅更多信息：[设置上下文窗口和预测范围的考虑因素](https://cloud.google.com/vertex-ai/docs/datasets/bp-tabular?hl=en#context-window)

In [ ]:
# Each row represents a day, so we set context and time horizon to 30 to represent 30 days.

CONTEXT_WINDOW = 30
TIME_HORIZON = 30

#### 运行训练流水线

通过调用带有以下参数的`run`方法来运行训练作业：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `target_column`：要作为标签进行训练的列名。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间。

完成`run`方法后返回`Model`资源。

#### 设置分层参数
我们希望按照‘产品’进行分组，以减少此级别的误差。
因此，您应将组参数设置为“product”。

将`group_total_weight`设置为非零重量意味着您想要根据个体损失来权衡组合聚合损失。为演示目的将其设置为10。

查看更多信息，请访问 https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/hierarchical

In [ ]:
time_column = "date"
time_series_identifier_column = "product_at_store"
target_column = "sales"

model = training_job.run(
    dataset=dataset_time_series,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=[
        "product_type",
        "product_category",
        "store",
        "product",
    ],
    forecast_horizon=TIME_HORIZON,
    data_granularity_unit="day",
    data_granularity_count=1,
    model_display_name="hierarchical_sales_forecasting_model",
    weight_column=None,
    hierarchy_group_columns=["product"],
    hierarchy_group_total_weight=10,
)

## 评估模型评分
在您的模型训练完成之后，您可以查看其评估分数。

In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

发送一批预测请求

现在您可以进行批量预测。

### 准备测试数据集

对于预测，测试数据集需要具有上下文窗口行，其中包含目标列的信息，以及目标列未知的随后时间范围行。构建这两个部分，并将它们合并成一个单独的 CSV 文件。

In [ ]:
import numpy as np

# Store start and end dates for context and horizon
date_context_window_start = date_cutoff
date_context_window_end = date_cutoff + np.timedelta64(CONTEXT_WINDOW, "D")
time_horizon_end = date_context_window_end + np.timedelta64(TIME_HORIZON, "D")

# Extract dataframes for context and horizon
df_test_context = df_test[
    (df_test["date"] >= date_context_window_start)
    & (df_test["date"] < date_context_window_end)
]
df_test_horizon = df_test[
    (df_test["date"] >= date_context_window_end) & (df_test["date"] < time_horizon_end)
].copy()

# Save a copy for validation of predictions
df_test_horizon_actual = df_test_horizon.copy()

# Remove sales for horizon (i.e. future dates)
df_test_horizon["sales"] = ""

# Write test data to CSV
df_test = pd.concat([df_test_context, df_test_horizon])
df_test.to_csv(DATASET_TEST_FILENAME, index=False)

# Save test dataset
DATASET_TEST_URI = f"{BUCKET_URI}/{DATASET_TEST_FILENAME}"

# Upload to GCS bucket
! gsutil cp {DATASET_TEST_FILENAME} {DATASET_TEST_URI}

### 检查上下文数据框

请注意销售列已填写。

In [ ]:
df_test_context.head()

### 检查时间范围数据框

注意销售列为空。

In [ ]:
df_test_horizon.head()

### 创建一个结果数据集

创建一个BigQuery数据集来存储预测结果。

In [ ]:
from google.cloud import bigquery

# Create client in default region
bigquery_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=aiplatform.initializer.global_config.credentials,
)

In [ ]:
def create_bigquery_dataset(name: str, region: str):
    batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(PROJECT_ID, name)

    bq_dataset = bigquery.Dataset("{}.{}".format(PROJECT_ID, name))

    dataset_region = region
    bq_dataset.location = dataset_region
    bq_dataset = bigquery_client.create_dataset(bq_dataset)
    print(
        "Created bigquery dataset {} in {}".format(
            batch_predict_bq_output_uri_prefix, dataset_region
        )
    )

    return batch_predict_bq_output_uri_prefix

In [ ]:
batch_predict_bq_output_uri_prefix = create_bigquery_dataset(
    name="hierarchical_forecasting_unique", region=REGION
)

### 发送批处理预测请求

您可以通过调用batch_predict()方法进行批处理预测，使用以下参数：

- `job_display_name`: 批处理预测作业的可读名称。
- `gcs_source`: 一个或多个批处理请求输入文件的列表。
- `bigquery_destination_prefix`: 存储批处理预测结果的BigQuery目标位置。
- `instances_format`: 输入实例的格式，可以是'bigquery'、'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`: 输出预测的格式，可以是'csv'、'jsonl'或'bigquery'。默认为'jsonl'。
- `machine_type`: 用于训练的机器类型。
- `accelerator_type`: 硬件加速器类型。
- `accelerator_count`: 要附加到工作复制品的加速器数量。
- `sync`: 如果设置为True，调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_prediction_job = model.batch_predict(
    job_display_name="hierarchical_forecasting_unique",
    gcs_source=DATASET_TEST_URI,
    instances_format="csv",
    bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
    predictions_format="bigquery",
    generate_explanation=True,
    sync=True,
)

### 查看批量预测结果

使用BigQuery Python客户端查询目标表，并将结果作为Pandas数据帧返回。

In [ ]:
# View the results as a dataframe
df_output = batch_prediction_job.iter_outputs(bq_max_results=1000).to_dataframe()

# Convert the dates to the datetime64 datatype
df_output["date"] = df_output["date"].astype("datetime64[ns]")

# Extract the predicted sales and convert to floats
df_output["predicted_sales"] = (
    df_output["predicted_sales"].apply(lambda x: x["value"]).astype(float)
)

df_output.head()

### 比较预测与实际值

绘制预测销量与实际值的图表

In [ ]:
import matplotlib.pyplot as plt

# Create a shared dataframe to plot predictions vs ground truth
df_output["sales_comparison"] = df_output["predicted_sales"]
df_output["is_ground_truth"] = False
df_test_horizon_actual["sales_comparison"] = df_test_horizon_actual["sales"]
df_test_horizon_actual["is_ground_truth"] = True
df_prediction_comparison = pd.concat([df_output, df_test_horizon_actual])

# Plot sales
fig = plt.gcf()
fig.set_size_inches(24, 12)

sns.relplot(
    data=df_prediction_comparison,
    x="date",
    y="sales_comparison",
    hue="product_at_store",
    style="store",
    row="is_ground_truth",
    height=5,
    aspect=4,
    kind="line",
    ci=None,
)

清理
要清理在此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud
项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 模型
- AutoML训练作业
- 批处理作业
- Cloud Storage存储桶

In [ ]:
from google.cloud import bigquery

# Create client in default region
bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=aiplatform.initializer.global_config.credentials,
)

In [ ]:
# Delete BigQuery datasets
bq_client.delete_dataset(
    f"{PROJECT_ID}.hierarchical_forecasting_unique",
    delete_contents=True,
    not_found_ok=True,
)

In [ ]:
# Delete Vertex AI resources
dataset_time_series.delete()
model.delete()
training_job.delete()
batch_prediction_job.delete()